In [19]:
from kubernetes.client import V1Container
from kserve import V1beta1InferenceService, V1beta1InferenceServiceSpec, V1beta1PredictorSpec
from kserve import constants
from kubernetes import client 
from kserve import KServeClient

In [23]:
name='tg-gcn-kserve-demo'
kserve_version='v1beta1'
namespace = 'kubeflow-user-example-com'
api_version = constants.KSERVE_GROUP + '/' + kserve_version

isvc = V1beta1InferenceService(api_version=api_version,
                                kind=constants.KSERVE_KIND,
                                metadata=client.V1ObjectMeta(
                                   name=name, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}),
                                   spec=V1beta1InferenceServiceSpec(
                                       predictor=V1beta1PredictorSpec(
                                           containers=[V1Container(image = "skirpichenko/kserve-base:latest", name = "kserve-base")]
                                       )
                                   )
                                )

In [24]:
KServe = KServeClient()
KServe.create(isvc)

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'},
  'creationTimestamp': '2022-05-27T12:45:52Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:sidecar.istio.io/inject': {}}},
     'f:spec': {'.': {}, 'f:predictor': {'.': {}, 'f:containers': {}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2022-05-27T12:45:48Z'}],
  'name': 'tg-gcn-kserve-demo',
  'namespace': 'kubeflow-user-example-com',
  'resourceVersion': '1670151',
  'uid': '28fde108-3a95-4431-92cb-d07f9657f7f1'},
 'spec': {'predictor': {'containers': [{'image': 'skirpichenko/kserve-base:latest',
     'name': 'kserve-container',
     'resources': {'limits': {'cpu': '1', 'memory': '2Gi'},
      'requests': {'cpu': '1', 'memory': '2Gi'}}}]}}}

In [25]:
KServe.get(name, namespace=namespace, watch=True, timeout_seconds=120)

NAME                 READY      PREV                      LATEST                    URL                                                              
tg-gcn-kserve-demo   Unknown                                                                                                                         
tg-gcn-kserve-demo   Unknown                                                                                                                         
tg-gcn-kserve-demo   Unknown    0                         100                                                                                        
tg-gcn-kserve-demo   Unknown    0                         100                                                                                        
tg-gcn-kserve-demo   Unknown    0                         100                                                                                        
tg-gcn-kserve-demo   Unknown    0                         100                                       

In [26]:
import requests

isvc_resp = KServe.get(name, namespace=namespace)
isvc_url = isvc_resp['status']['address']['url']

print(isvc_url)

inference_input = {
    "nodes": [
      {"primary_id": 7, "type": "Paper"}, 
      {"primary_id": 999, "type": "Paper"}
    ]
}

response = requests.post(isvc_url, json=inference_input)
print(response.text)

http://tg-gcn-kserve-demo.kubeflow-user-example-com.svc.cluster.local/v1/models/tg-gcn-kserve-demo:predict
{"predictions": [{"primary_id": "7", "label": 3}, {"primary_id": "999", "label": 2}]}
